# Question the database

In [84]:
from openai import OpenAI
import sys
import tiktoken
import requests
import getpass
import urllib
from urllib.parse import urlencode

email = None
url = 'https://testapi.xbrl.us/oauth2/token'

In [85]:
# Input setup
if email is None: 
    email = input("Enter your username (email).")
    password = getpass.getpass(prompt='Password: ')
    clientid = getpass.getpass(prompt='Client ID: ')
    secret = getpass.getpass(prompt='Secret: ')

body_auth = {'username' : ''.join(email), 
            'client_id': ''.join(clientid), 
            'client_secret' : ''.join(secret), 
            'password' : ''.join(password), 
            'grant_type' : 'password', 
            'platform' : 'ipynb' }


payload = urlencode(body_auth)
headers = {"Content-Type": "application/x-www-form-urlencoded"}
res = requests.request("POST", url, data=payload, headers=headers)
auth_json = res.json()

if 'error' in auth_json:
    print ("\n\nThere was a problem generating an access token with these credentials. Run the first cell again to enter credentials.")
else:
    access_token = auth_json['access_token']
    refresh_token = auth_json['refresh_token']
    newaccess = newrefresh = ''
    #password = secret = None
    print ("\n\nYour access token expires in 60 minutes. After it expires, run this cell again.")


marc.ward@xbrl.us


Your access token expires in 60 minutes. After it expires, run the cell immediately below this one to generate a new token and continue to use the query cell. 

For now, skip ahead to the section 'Make a Query'.


In [118]:
def get_embedding(value):
    client = OpenAI()

    return client.embeddings.create(input=value, model='text-embedding-3-small').data[0].embedding


def get_content(question):
    search_endpoint = 'https://testapi.xbrl.us/api/v1/query/getcontent'
    params = {"question" : str(question)}

    res = requests.post(search_endpoint, data=params, headers={'Authorization' : 'Bearer {}'.format(access_token)})

    return res.text


def answer_question(question, content):

    tokenizer = tiktoken.get_encoding("cl100k_base")
    client = OpenAI()
    max_len = 8192

    while True:
        if len(tokenizer.encode(content)) <= max_len:
            break
        else: 
            content = content[:int(len(content) * .9)]

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages = [
                {"role": "system", "content": "Answer the question based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\n"},
                {"role": "user", f"content": f"Context: {content}\n\n---\n\nQuestion: {question}\nAnswer:"}
            ],
            max_tokens=150,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None        
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        return ""  
    

def refresh_token():
    token = token if newrefresh != '' else refresh_token 

    refresh_auth = {'client_id': ''.join(clientid), 
                'client_secret' : ''.join(secret), 
                'grant_type' : 'refresh_token', 
                'platform' : 'ipynb', 
                'refresh_token' : ''.join(token) }
    refreshres = requests.post(url, data=refresh_auth)
    refresh_json = refreshres.json()
    access_token = refresh_json['access_token']
    refresh_token = refresh_json['refresh_token']#print('access token: ' + access_token + 'refresh token: ' + refresh_token)
    print('Your access token is refreshed for 60 minutes. If it expires again, run this cell to generate a new token and continue to use the query cells below.')
   

    
def send_query(search_endpoint):
    params = {}



    count = 0
    query_start = datetime.now()
    printed = False
    while True:
        if not printed:
            print("On", query_start.strftime("%c"), email, "(client ID:", str(clientid.split('-')[0]), "...) started the query and")
            printed = True
        res = requests.get(search_endpoint, params=params, headers={'Authorization' : 'Bearer {}'.format(access_token)})
        res_json = res.json()
        if 'error' in res_json:
            print('There was an error: {}'.format(res_json['error_description']))
            break

        print("up to", str(offset_value + res_json['paging']['limit']), "records are found so far ...")

        res_df += res_json['data']

        if res_json['paging']['count'] < res_json['paging']['limit']:
            print(" - this set contained fewer than the", res_json['paging']['limit'], "possible, only", str(res_json['paging']['count']), "records.")
            break
        else: 
            offset_value += res_json['paging']['limit'] 
            if 100 == res_json['paging']['limit']:
                    params['fields'] = orig_fields + ',fact.offset({})'.format(offset_value)
                    if offset_value == 10 * res_json['paging']['limit']:
                            break 
            elif 500 == res_json['paging']['limit']:
                    params['fields'] = orig_fields + ',fact.offset({})'.format(offset_value)
                    if offset_value == 4 * res_json['paging']['limit']:
                            break 
            params['fields'] = orig_fields + ',fact.offset({})'.format(offset_value)

    if not 'error' in res_json:
        current_datetime = datetime.now().replace(microsecond=0)
        time_taken = current_datetime - query_start
        index = pd.DataFrame(res_df).index
        total_rows = len(index)
        your_limit = res_json['paging']['limit']
        limit_message = "If the results below match the limit noted above, you might not be seeing all rows, and should consider upgrading (https://xbrl.us/access-token).\n"
        
        if your_limit == 100:
            print("\nThis non-Member account has a limit of " , 10 * your_limit, " rows per query from our Public Filings Database. " + limit_message)
        elif your_limit == 500:
            print("\nThis Basic Individual Member account has a limit of ", 4 * your_limit, " rows per query from our Public Filings Database. " + limit_message)
        
        print("\nAt " + current_datetime.strftime("%c") +  ", the query finished with  ", str(total_rows), "  rows returned in " + str(time_taken) + " for \n" +  urllib.parse.unquote(res.url))
        


In [124]:
# Ask Questions

while True:
    question = input("Enter your question: ")
    if question.lower() == 'exit':
        print('quitting...')
        sys.exit()
        
    ques_embedded = get_embedding(question)
    info = get_content(ques_embedded)

    answer = answer_question(question, info)
    print("\nQUESTION:\n{}\n\nANSWER: \n{}\n\n".format(question, answer))
    print("")






QUESTION:
how much in prepaid expensis did the company "ParkerVision" declare?

ANSWER: 
$61 million




QUESTION:
when did the company "ParkerVision" declare $61 million in prepaid assets?

ANSWER: 
December 31, 2023



quitting...


SystemExit: 

C:\Users\Marc\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
